In [15]:
# Load data
with open("../data/day13.txt", "r") as f:
    input = f.read()

data = input.split("\n")
timestamp_earliest = int(data[0])
lines = [int(value) for value in data[1].split(",") if value != "x"]
lines
#timestamp_earliest = 939
#lines = [7,13,"x","x",59,"x",31,19]

[29, 41, 601, 23, 13, 17, 19, 463, 37]

In [16]:
# Puzzle 1
import numpy as np

dts = [line - (timestamp_earliest % line) for line in lines]
min(dts) * lines[np.argmin(dts)]

4808

In [215]:
# Puzzle 2

# Load data
with open("../data/day13.txt", "r") as f:
    input = f.read()
data = input.split("\n")
bus_ids = data[1].split(",")

line_list = [(int(value), m) for value, m in zip(bus_ids, range(len(bus_ids))) if value != "x"]
print(line_list)
M = len(line_list)

[(29, 0), (41, 19), (601, 29), (23, 37), (13, 42), (17, 46), (19, 48), (463, 60), (37, 97)]


In [216]:
# https://en.wikipedia.org/wiki/Diophantine_equation
# Approach it as a System of linear Diophantine equations, AX = C, with X to be calculated. We end up with a free parameter that needs to be set to get the lowest value of the solution timestamp that is larger than zero. A bit annoying because it is a big number. Luckily, the solution is monotonous with this number, so we can do some quick binary search like optimization. What I thought would be an elegant solution turned out to be quite the hack-job...

from smithnormalform import matrix, snfproblem, z

# Initialize with zeros
A = matrix.Matrix(M, M + 1, [z.Z(0)] * M * (M + 1))
C = matrix.Matrix(M, 1, [z.Z(0)] * M)

# Set with values
m = 0
for value in line_list:
    A.set(m, 0, z.Z(1))
    A.set(m, m+1, z.Z(-value[0]))
    C.set(m, 0, z.Z(-value[1]))
    m += 1
print(A)
print(C)

[1 -29 0 0 0 0 0 0 0 0]
[1 0 -41 0 0 0 0 0 0 0]
[1 0 0 -601 0 0 0 0 0 0]
[1 0 0 0 -23 0 0 0 0 0]
[1 0 0 0 0 -13 0 0 0 0]
[1 0 0 0 0 0 -17 0 0 0]
[1 0 0 0 0 0 0 -19 0 0]
[1 0 0 0 0 0 0 0 -463 0]
[1 0 0 0 0 0 0 0 0 -37]

[0]
[-19]
[-29]
[-37]
[-42]
[-46]
[-48]
[-60]
[-97]



In [220]:
# Calculate Smith normal form of A
prob = snfproblem.SNFProblem(A)
prob.computeSNF()
assert prob.isValid()
U = prob.S # m*m, 3*3
V = prob.T # n*n, 4*4
B = U * A * V
D = U * C
if False:
    print(A)
    print(U)
    print(V)
    print(B)
    print(D) 

# The result is vector y, but this has a free parameter at the last position, that we need to set such that all values are positive. This is a bit annoying as n is a large integer.
def problem(n):
    """Return solution time for free input parameter (last element of vector y)"""
    y = matrix.Matrix(M + 1, 1, [*[D.get(m,0).get_q(B.get(m,m)) for m in range(M)], z.Z(int(n))])
    X = V * y
    #print(y)
    #print(X)
    assert A * X == C # Check solution
    solution = X.get(0,0).a
    return solution

problem(0)

-4882889813696346603443573187684113619967770664400692368031005197179627

In [222]:
# Find value of n for which problem(n) is closest to zero, but not <0. Sort of binary search?

n_low = 0 # initial value that gives negative result
n_high = 1e55 # initial value that gives positive result
test = problem(n)
for k in range(0,200):
    test_low = problem(n_low)
    test_high = problem(n_high)
    assert test_low < 0
    assert test_high > 0

    n_new = n_low + int(np.ceil((n_high - n_low)/2)) # new guess is in between previous numbers
    test_new = problem(n_new)
    print(test_new)

    if n_new in [n_low, n_high]:
        print("Done!", k)
        print(n_low, n_high, n_new, test_new)
        break

    if test_new > 0:
        n_high = n_new
    if test_new < 0:
        n_low = n_new


1028406868322368457058944530296130354556873855003174935806262514050325
-1927241472686989073192314328693991632705448404698758716112371341564651
-449417302182310308066684899198930639074287274847791890153054413757163
289494783070029074496129815548599857741293290077691522826604050146581
-79961259556140616785277541825165390666496992385050183663225181805291
104766761756944228855426136861717233537398148846320669581689434170645
12402751100401806035074297518275921435450578230635242959232126182677
-33779254227869405375101622153444734615523207077207470351996527811307
-10688251563733799670013662317584406590036314423286113696382200814315
857249768334003182530317600345757422707131903674564631424962684181
-4915500897699898243741672358619324583664591259805774532478619065067
-2029125564682947530605677379136783580478729678065604950526828190443
-585937898174472174037679889395513078885798887195520159550932753131
135655935079765504246318855475122171910666508239522235937014965525
-22514098154735333489568051

In [207]:
print(problem(n_new-1))
print(problem(n_new)) # Solution
print(problem(n_new+1))

-440514293298069
741745043105674
1924004379509417
